# DC Height, Weight, & Gender

## Author: Oliver Gladfelter

## Date: Sep 6, 2018

In [1]:
import lxml.html as lh
import io
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats

# Data Collection
## All 26 Marvel Category Wiki Pages Containing Character Links
Each of these category pages contain a list of links to 200 Marvel characters, for a total of 4,984 characters. These are all of the characters in the Marvel Wiki for which a weight is listed. To access all ~5k, we need to iterate over all 26 pages to collect links

In [2]:
allWeightPages = [
    'http://dc.wikia.com/wiki/Category:Weight?from=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+120%0AShawna+Baez+%28Arrow%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+130%0AWilliam+Batson+%28Earth-S%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+145%0AJaime+Reyes+%28New+Earth%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+160%0AThom+Kallor+%28Earth-Prime%29&subcatfrom=00100&filefrom=00100',   
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+173%0AEvan+McCulloch+%28New+Earth%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+180%0AMusketeer+%28New+Earth%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+190%0AThomas+Bolt+%28New+Earth%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+205%0AChristopher+Smith+%28New+Earth%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+230%0AHightower+%28Wildstorm+Universe%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+320%0AM%27Dahna+%28Green+Lantern+Movie%29&subcatfrom=00100&filefrom=00100',
    'http://dc.wikia.com/wiki/Category:Weight?pagefrom=Weight+92%0AMara+%28New+Earth%29&subcatfrom=00100&filefrom=00100']

## Function to Web Scrape A Single Marvel Category Wiki Page
This function accepts a single URL - to a Marvel category page - and returns a list of links to every character wiki page listed in the category page

In [9]:
def getLinksFromCategoryPage(url):

    r = requests.get(url)

    if r.status_code != 200:
        print("Error")
    else:
        soup = BeautifulSoup(r.content, "lxml")

    # returns the last three unordered lists of the webpage
    unorderedLists = soup.find_all("ul",{'class':''})[1:]
    
    # the base for every character's page URL
    linkBase = 'http://dc.wikia.com'
    links = []

    # for each unordered list (total of 3), grab the list of characters and save the wiki link to each one
    for unorderedList in unorderedLists:
        listOfCharacters = unorderedList.find_all('li')
        for character in listOfCharacters:
            links.append(linkBase + character.find('a')['href'])
            
    return links

## Function to Web Scrape Marvel Character Pages
When given a list of links to character pages, this function will scrape the gender, height, weight, and first appearance of each character and return a data frame containing a row for each character

In [5]:
def getDataFromCharacterPage(listOfURLs):
    
    linkList = []
    genderList = []
    heightList = []
    weightList = []
    firstAppearanceList = []

    for link in listOfURLs:

        r = requests.get(link)

        if r.status_code != 200:
            print("Error")
        else:
            soup = BeautifulSoup(r.content, "lxml")

        # return an array of each characteristic banner, which includes a measure name and value in one string
        stats = soup.findAll("div",{"class": "pi-item pi-data pi-item-spacing pi-border-color"})

        gender = ''
        height = ''
        weight = ''

        # regEx to get only the vital info
        for stat in stats:
            if 'Gender' in stat.text:
                if re.search('Gender\n(.{,})',stat.text) is not None:
                    gender = re.search('Gender\n(.{,})',stat.text).group(1)
            if 'Height' in stat.text:
                if re.search('Height\n(.{,})\n',stat.text) is not None:
                    height = re.search('Height\n(.{,})\n',stat.text).group(1)
            if 'Weight' in stat.text:
                if re.search('Weight\n(.{,})\n',stat.text) is not None:
                    weight = re.search('Weight\n(.{,})\n',stat.text).group(1)

        linkList.append(link)
        genderList.append(gender)
        heightList.append(height)
        weightList.append(weight)
        firstAppearanceList.append(soup.find("td",{"class": "pi-horizontal-group-item pi-data-value pi-font pi-border-color pi-item-spacing"}).text)

    return pd.DataFrame({'link':linkList,'gender':genderList,'height':heightList,'weight':weightList,'firstAppearance':firstAppearanceList})

## Loop to Web Scrape All Category Pages To Collect Character Page Links
We now run the getLinksFromCategoryPage() 26 times, one for each category page. All links will be saved to one list.

In [10]:
CharacterPageLinks = []

for page in allWeightPages:
    CharacterPageLinks = CharacterPageLinks + getLinksFromCategoryPage(page)

In [11]:
len(CharacterPageLinks) # Should be 2,218

1477

## Scrape all 5,000 Character Pages for Weight, Height, Gender Data

In [13]:
data = getDataFromCharacterPage(CharacterPageLinks)

In [15]:
data.to_csv("C:\\Users\\glol7001\\Documents\\DCdata.csv")

In [9]:
data = pd.read_csv("MarvelData.csv")
del data['Unnamed: 0']

# Data Cleaning

## Removing characters who don't have all variables of interested listed on their Wiki page

In [17]:
data2 = data[(data['gender'] != '') & (data['height'] != '') & (data['weight'] != '')]
data2 = data2.reset_index()
del data2['index']

## Parsing Out First Appearance Dates From The 'firstAppearance' Column
The 'firstAppearance' value includes both the first publication in which the character was introduced as well as a date (in varying formats). It is the date, particularly the year, that we are interested in.

In [19]:
def getDate(value):
    if re.search('.{,}\((.{3,14}\s?\d{4})\)',value) is not None:
        return re.search('.{,}\((.{3,14}\s?\d{4})\)',value).group(1)
    elif re.search('\d{4}',value) is not None:
        return re.search('\d{4}',value).group(0)
    else:
        return 0
    
# this will result in a few 0s, where the first appearance date was missing from the character's page
data2['firstAppearanceDate'] = data2['firstAppearance'].apply(getDate)

In [20]:
data2.tail(3)

,link,gender,height,weight,firstAppearance,firstAppearanceDate
1460,http://dc.wikia.com/wiki/Thaddeus_Sivana_(Eart...,Male,"5' 2""",98 lbs (44 kg),"Whiz Comics #2(February, 1940)","February, 1940"
1461,http://dc.wikia.com/wiki/Cyrus_Gold_(New_Earth),Male,"6' 7"" [2]",980 lbs (445 kg) [2],"All-American Comics #61(October, 1944)","October, 1944"
1462,http://dc.wikia.com/wiki/Imperiex-Prime_(New_E...,Male,"6' 7""",986 lbs (447 kg),"Superman Vol 2 #153(February, 2000)","February, 2000"


## Filling In Missing Release Dates
A publication date wasn't listed in some cases, but the name of the introducing medium (comic book, TV, video game, etc) always is. In those cases, we need to scrape the comic book on Marvel Wiki to get the publication date.

In [21]:
missingDates = data2[data2['firstAppearanceDate'] == 0]
missingDates = missingDates.reset_index()
del missingDates['index']

In [36]:
dates = ['2012',-1,-1,'2001','1989','2015',-1,'1988',-1,'1993','1993','2011']

for num in range(0,len(missingDates)):
    missingDates['firstAppearanceDate'][num] = dates[num]
    
missingDates = missingDates[missingDates['firstAppearanceDate'] != -1]

In [37]:
missingDates.tail(3)

,link,gender,height,weight,firstAppearance,firstAppearanceDate
9,http://dc.wikia.com/wiki/Percival_Edmund_Chang...,Male,"5' 3""",250 lbs (113 kg),Deathmate Black,1993
10,http://dc.wikia.com/wiki/Caitlin_Fairchild_(Wi...,Female,"6' 4""",300 lbs (136 kg),Deathmate Black,1993
11,http://dc.wikia.com/wiki/Chaselon_(Green_Lante...,Male,"10' 0""",720 lbs (327 kg),[[Green Lantern (Movie)]],2011


## Reattaching Updated Rows To Main Dataframe

In [38]:
notMissingDates = data2[data2['firstAppearanceDate'] != 0]
data3 = pd.concat([notMissingDates,missingDates])
data3 = data3.reset_index()
del data3['index']

In [39]:
data3 = data3.dropna()

## Derive Normalized Year, Gender, Height, and Weight Columns

In [ ]:
def getYear(value):
    return int(re.search('\d{4}',value).group(0))
    
data3['yearFirstAppeared'] = data3['firstAppearanceDate'].apply(getYear)

In [41]:
def MaleFemale(value):
    if (value != 'Male') and (value != 'Female'):
        return -1
    else:
        return value

data3['gender'] = data3['gender'].apply(MaleFemale)

# Removing characters where gender is either missing or listed as agender or fluid, for analytical purposes
data3 = data3[data3['gender'] != -1]
data3 = data3.reset_index()
del data3['index']

In [42]:
def calcHeight(value):
    
    if re.search('(\d)[\',’]\s?(\d{1,2})',value) is None:
        return -1
    
    feet = int(re.search('(\d)[\',’]\s?(\d{1,2})',value).group(1))
    inch = int(re.search('(\d)[\',’]\s?(\d{1,2})',value).group(2))
    return (feet * 12) + inch

data3['heightInInches'] = data3['height'].apply(calcHeight)

In [ ]:
# Some manual fixes 
data3['heightInInches'][196] = 67
data3['heightInInches'][1203] = 84
data3['heightInInches'][1213] = 125
data3['heightInInches'][1309] = 72
data3['heightInInches'][1316] = 74
data3['heightInInches'][1367] = 15
data3['heightInInches'][1415] = 86

In [49]:
def calcWeight(value):
    
    if re.search('\d{1,3}',value) is None:
        return -1

    return int(re.search('\d{1,3}',value).group(0))

data3['weightInPounds'] = data3['weight'].apply(calcWeight)

In [6]:
data3 = pd.read_csv("MarvelData9-6-18.csv", encoding = 'latin')
del data3['Unnamed: 0']

## Parsing 'link' Column For Character Name and Universe

In [ ]:
def getName(value):
    if re.search('wikia.com/wiki/(.{,})_\((.{,})\)',value) is None:
        return -1
    return re.search('wikia.com/wiki/(.{,})_\((.{,})\)',value).group(1).replace("_"," ")

def getUniverse(value):
    if re.search('wikia.com/wiki/(.{,})_\((.{,})\)',value) is None:
        return -1
    return re.search('wikia.com/wiki/(.{,})_\((.{,})\)',value).group(2).replace("_"," ")

data3['name'] = data3['link'].apply(getName)
data3['universe'] = data3['link'].apply(getUniverse)

data3['name'][915] = 'Telos'
data3['name'][1210] = 'Santa Claus'
data3['name'][1239] = 'Clark Savage, Jr.'

data3 = data3[data3['name'] != -1]
data3 = data3.reset_index()
del data3['index']

In [61]:
data3.tail(2)

,link,gender,height,weight,firstAppearance,firstAppearanceDate,yearFirstAppeared,heightInInches,weightInPounds,name,universe
1454,http://dc.wikia.com/wiki/Caitlin_Fairchild_(Wi...,Female,"6' 4""",300 lbs (136 kg),Deathmate Black,1993,1993,76,300,Caitlin Fairchild,Wildstorm Universe
1455,http://dc.wikia.com/wiki/Chaselon_(Green_Lante...,Male,"10' 0""",720 lbs (327 kg),[[Green Lantern (Movie)]],2011,2011,0,720,Chaselon,Green Lantern Movie


In [62]:
data3.to_csv("C:\\Users\\glol7001\\Documents\\DCDataComplete.csv")